## Imports

In [1]:
import os

from plio.io import io_controlnetwork
from knoten.csm import create_csm
from scipy import sparse
import ale
import csmapi
import numpy as np

import matplotlib.pyplot as plt

from knoten.bundle import *

## Load Network and Generate Sensors

In [ ]:
cubes = '/scratch/csm2020/data/cubes2.lis'
sensors = generate_sensors(cubes)

network = '/scratch/csm2020/data/hand_dense.net'
cnet = io_controlnetwork.from_isis(network)
cnet = compute_apriori_ground_points(cnet, sensors)

## Determine Which Sensor Parameters to Solve For

In [ ]:
all_parameters = {sn: get_sensor_parameters(sensor) for sn, sensor in sensors.items()}
for sn, parameters in all_parameters.items():
    print(f"Image: {sn}")
    for param in parameters:
        print(f"  {param.name} | {param.index} | {param.value}")

## Solve for angles and angular rates

In [ ]:
solve_parameters = {sn: params[6:12] for sn, params in all_parameters.items()}

## Compute the Column Indices for Parameters

In [ ]:
column_dict = compute_coefficient_columns(cnet, sensors, solve_parameters)

## Compute the Weight Matrix
#### According to the weighted Normal equation (J.TWJ), W needs to be a square matrix the size of (# of measures)x2. So it is the weight of the observations. In ISIS, the weight of the observations are an inverted function of the size of the pixels on the focal plane (resolution). However, in csm we do not have access to that information. 
#### For the time being, since we are working exclusively with CTX images we are going to set the weight matrix equal to the identity matrix -> all observations have the same weight.

In [ ]:
num_observations = 2 * len(cnet)
W_observations = np.eye(num_observations) # this is a place holder until Jesse adds his calculations
W_params = compute_parameter_weights(cnet, sensors, solve_parameters, column_dict)

## Whole (Dense) Bundle Process in a loop with LM terms

The next three cells show examples of the bundle adjustment loop running with Levenberg-Marquat added.

This example shows the first heuristic for initializing $\lambda$, the weighting parameter, from Numerical Recipes in C++

NumericalRecipes in C++ recommended starting with $\lambda$ = 0.01, and then increasing or decreasing this value by an order of magnitude each iteration depending on whether the last iteration increased or decreased 
$\sigma_0$.

Step sizes will get smaller as the target is approached.

### LM using Numerical Recipies's heuristic

$\lambda = 0.01$ to start out, then increased or decreased by an order of magnitude each step. 

`damping` is used as the variable for $\lambda$, since in python `lambda` is a reserved word.

In [ ]:
## sensors = generate_sensors(cubes) # generate sensors
cnet = io_controlnetwork.from_isis(network) # load in network
cnet = compute_apriori_ground_points(cnet, sensors) # calculate ground points

### INPUTS ###
all_parameters = {sn: get_sensor_parameters(sensor) for sn, sensor in sensors.items()} #all parameters
parameters = {sn: parameter[6:12] for sn, parameter in all_parameters.items()} #just solving for camera angles and angle velocity
##############

column_dict = compute_coefficient_columns(cnet, sensors, parameters)
num_parameters = max(col_range[1] for col_range in column_dict.values())
num_observations = 2 * len(cnet)
W_observations = np.eye(num_observations)
W_params = compute_parameter_weights(cnet, sensors, parameters, column_dict)

iteration = 0
V = compute_residuals(cnet, sensors)
dX = np.zeros(W_params.shape[0]) #initialize for sigma calculatio
sigma0 = compute_sigma(V, dX, W_params, W_observations)
print(f'iteration {iteration}: sigma0 = {sigma0}\n')

max_iterations = 100
tol = 1e-10
total_correction = np.zeros(num_parameters)
damping = 0.01
for i in range(max_iterations):   
    iteration += 1
    old_sigma0 = sigma0
    
    J = compute_jacobian(cnet, sensors, parameters, column_dict)    
#    C = J.T.dot(W_observations).dot(V) - damping*np.identity(W_params.shape[0]).dot(total_correction) - W_params.dot(total_correction)
    C = J.T.dot(W_observations).dot(V) - W_params.dot(total_correction)
    N = J.T.dot(W_observations).dot(J) + damping*np.identity(W_params.shape[0]) + W_params
    dX = np.linalg.inv(N).dot(C) # calculate change in camera parameters and ground points
    
    total_correction += dX
    print(f'corrections: mean = {dX.mean()} min = {dX.min()} max = {dX.max()}')
    
    update_parameters(sensors, parameters, cnet, dX, column_dict)
    
    V = compute_residuals(cnet, sensors)
    sigma0 = compute_sigma(V, dX, W_params, W_observations)
    if (sigma0 < old_sigma0):
        damping*=0.1
    else:
        damping*=10
        
    print(f'iteration {iteration}: sigma0 = {sigma0}\n')
    
    if (abs(sigma0 - old_sigma0) < tol):
        print(f'change in sigma0 of {abs(sigma0 - old_sigma0)} converged!')
        break
    

### sba/levmar's heuristic

$\lambda$ initially set to a user supplied value (or default = 0.001) * the max value in $J^{T}J$

Updates either:

(1) $\lambda = max(1/3, 1-(2\rho -1)^3) , \nu = 2.0$

(2) $\lambda = \lambda\nu, \nu = 2.0\nu$

See: http://users.ics.forth.gr/~argyros/mypapers/2004_08_tr340_forth_sba.pdf

In [ ]:
## sensors = generate_sensors(cubes) # generate sensors
cnet = io_controlnetwork.from_isis(network) # load in network
cnet = compute_apriori_ground_points(cnet, sensors) # calculate ground points

### INPUTS ###
all_parameters = {sn: get_sensor_parameters(sensor) for sn, sensor in sensors.items()} #all parameters
parameters = {sn: parameter[6:12] for sn, parameter in all_parameters.items()} #just solving for camera angles and angle velocity
##############

column_dict = compute_coefficient_columns(cnet, sensors, parameters)
num_parameters = max(col_range[1] for col_range in column_dict.values())
num_observations = 2 * len(cnet)
W_observations = np.eye(num_observations)
W_params = compute_parameter_weights(cnet, sensors, parameters, column_dict)

iteration = 0
V = compute_residuals(cnet, sensors)
dX = np.zeros(W_params.shape[0]) #initialize for sigma calculatio
sigma0 = compute_sigma(V, dX, W_params, W_observations)
print(f'iteration {iteration}: sigma0 = {sigma0}\n')

max_iterations = 25
tol = 1e-10
total_correction = np.zeros(num_parameters)
damping = 0.001
nu = 2.0
for i in range(max_iterations):   
    iteration += 1
    old_sigma0 = sigma0
    
    J = compute_jacobian(cnet, sensors, parameters, column_dict)
    if i==0:
        damping *= np.max(J.T.dot(J))
    
#    C = J.T.dot(W_observations).dot(V) - damping*np.identity(W_params.shape[0]).dot(total_correction) - W_params.dot(total_correction)
    C = J.T.dot(W_observations).dot(V) - W_params.dot(total_correction)
    N = J.T.dot(W_observations).dot(J) + damping*np.identity(W_params.shape[0]) + W_params
    dX = np.linalg.inv(N).dot(C) # calculate change in camera parameters and ground points
    
    total_correction += dX
    print(f'corrections: mean = {dX.mean()} min = {dX.min()} max = {dX.max()}')
    
    update_parameters(sensors, parameters, cnet, dX, column_dict)
    
    old_V = V
    V = compute_residuals(cnet, sensors)
    sigma0 = compute_sigma(V, dX, W_params, W_observations)
    if (sigma0 < old_sigma0):
        damping = max(1.0/3.0, (np.linalg.norm(old_V)**2 - np.linalg.norm(V)**2)/(dX.T.dot(damping*dX + C)))
        nu = 2
    else:
        damping = damping * nu
        nu = 2 * nu

    print(f'iteration {iteration}: sigma0 = {sigma0}\n')
    
    if (abs(sigma0 - old_sigma0) < tol):
        print(f'change in sigma0 of {abs(sigma0 - old_sigma0)} converged!')
        break
    